In [1]:
import numpy as np
import random
from tqdm import tqdm
import matplotlib.pyplot as plt
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torch
import wandb

random.seed(27)
np.random.seed(27)
plt.style.use('ggplot')

# Section 1: Pilot study of subset, used to find feasible hyperparameters

### Dataset - A (1/10) subset of cifar

We will choose a subset of the cifar10, and make sure there is class balance

In [2]:
ds = "cifar10"
transform = transforms.Compose([transforms.ToTensor()])

if ds == "cifar10":
    train_ds = datasets.CIFAR10(root="./data", train=True, download=True, transform=transform)
    test_ds = datasets.CIFAR10(root="./data", train=False, download=True, transform=transform)

    X_train = train_ds.data.astype(np.float32) / 255.0   
    y_train = np.array(train_ds.targets, dtype=np.int64)

    X_test = test_ds.data.astype(np.float32) / 255.0
    y_test = np.array(test_ds.targets, dtype=np.int64)

    # flatten to (N, D)
    X_train = X_train.reshape(X_train.shape[0], -1)
    X_test = X_test.reshape(X_test.shape[0], -1)

elif ds == "mnist":
    mnist_train = datasets.MNIST(root="./data", train=True, download=True)
    mnist_test = datasets.MNIST(root="./data", train=False, download=True)
    X_train = mnist_train.data.numpy().astype(np.float32) / 255.0
    y_train = mnist_train.targets.numpy().astype(np.int64)
    X_test  = mnist_test.data.numpy().astype(np.float32) / 255.0
    y_test  = mnist_test.targets.numpy().astype(np.int64)
    X_train = X_train.reshape(X_train.shape[0], -1)
    X_test  = X_test.reshape(X_test.shape[0], -1)



subset_ratio = 0.2      #20% of each class
num_classes_total = 10  
num_classes = 3         

def balanced_subset(X, y, ratio, classes_to_take):
    indices = []
    for c in range(classes_to_take):
        class_idx = np.where(y == c)[0]
        n_samples = max(1, int(len(class_idx) * ratio))
        chosen = np.random.choice(class_idx, n_samples, replace=False)
        indices.append(chosen)
    indices = np.concatenate(indices)
    np.random.shuffle(indices)
    return X[indices], y[indices]

X_train, y_train = balanced_subset(X_train, y_train, subset_ratio, num_classes)
X_test,  y_test  = balanced_subset(X_test,  y_test,  subset_ratio, num_classes)

def one_hot_encode(y, num_classes):
    y = np.asarray(y, dtype=int)
    m = y.size
    Y = np.zeros((num_classes, m), dtype=float)
    Y[y, np.arange(m)] = 1.0
    return Y

Y_train = one_hot_encode(y_train, num_classes)
Y_test = one_hot_encode(y_test, num_classes)

print("Shapes:", X_train.shape, Y_train.shape, X_test.shape, Y_test.shape)

Shapes: (3000, 3072) (3, 3000) (600, 3072) (3, 600)


### Pilot study - we search for a feasible range of hyperparameters using sweeps

In [3]:
from NN.denseLayer import *
from NN.loss import *
from NN.optimizer import *
from NN.activations import *

> Setup WANDB tracker, where we will perform a simple grid search of a VERY broad search

In [4]:
import matplotlib.pyplot as plt
import os
import wandb
from getAPI import retrieveApi
from sklearn.model_selection import KFold

os.environ['WANDB_API_KEY'] = retrieveApi()

> Do hyperparameter sweep, on a very small subset of cifar-10

The code is based on the tutorials given in the sweeps documentation:
- https://docs.wandb.ai/models/sweeps/define-sweep-configuration#python-script-or-notebook
- https://docs.wandb.ai/models/tutorials/sweeps

We are doing a random search, where we try and get an idea of what values work and what does not, maybe we can rule out some ranges, which will make the coarse hyperparameter search better

In [5]:
def accuracy_from_probs(A, Y_true_onehot):
    preds = A.argmax(axis=0)
    truths = Y_true_onehot.argmax(axis=0)
    return float((preds == truths).mean())

def get_activation_instance(name):
    if name == "relu":
        return ReLU()
    elif name == "tanh":
        return Tanh()
    elif name == "sigmoid":
        return Sigmoid()

def get_optimizer(name, lr):
    if name == "sgd":
        return SGD(learning_rate=lr)
    elif name == "adam":
        return Adam(learning_rate=lr)


def build_layers(input_dim, cfg):

    layers = []
    prev = input_dim

    for _ in range(int(cfg['num_hidden_layers'])):
        act = get_activation_instance(cfg['activation'])
        layers.append(DenseLayer(prev, int(cfg['n_hidden_units']), activation=act,
                                 initializer=HeInitializer(), l2_coeff=float(cfg.get('l2_coeff', 0.0))))
        prev = int(cfg['n_hidden_units'])

    # final layer with Softmax and small-normal initializer
    layers.append(DenseLayer(prev, int(cfg['num_classes']), activation=Softmax(),
                             initializer=NormalInitializer(mean=0.0, std=0.01), l2_coeff=0.0))
    return layers


sweep_configuration = {
    "method": "random",
    "metric": {"name": "mean_cv_accuracy", "goal": "maximize"},
    "parameters": {
        "learning_rate": {"distribution": "uniform", "min": 1e-6, "max": 1.0},
        "batch_size": {"values": [32, 64, 128]},
        "epochs": {"value": 30},
        "num_hidden_layers": {"distribution": "int_uniform", "min": 1, "max": 5},
        "n_hidden_units": {"distribution": "int_uniform", "min": 32, "max": 128},
        "l2_coeff": {"distribution": "uniform", "min": 0.0, "max": 0.001},
        "optimizer": {"values": ["sgd", "adam"]},           
        "activation": {"values": ["relu", "tanh", "sigmoid"]},
        "num_classes": {"value": 3},                 
    },
}

# create the sweep (do this once; it returns a sweep_id)
sweep_id = wandb.sweep(sweep=sweep_configuration, project="numpy_nn")



def train_on_split(X_train_fold, Y_train_fold, X_val_fold, Y_val_fold, cfg):
    """
    Train a model on one train/val split.
    Returns final validation accuracy.
    """

    layers = build_layers(X_train_fold.shape[1], cfg)
    optimizer = get_optimizer(cfg['optimizer'], float(cfg['learning_rate']))
    loss_fn = CrossEntropyLoss()

    m = X_train_fold.shape[0]
    epochs = int(cfg["epochs"])
    batch_size = int(cfg["batch_size"])

    for epoch in range(epochs):
        perm = np.random.permutation(m)

        Xs = X_train_fold[perm].T     # (in_dim, m)
        Ys = Y_train_fold[:, perm]    # (num_classes, m)

        for i in range(0, m, batch_size):
            Xb = Xs[:, i:i+batch_size]
            Yb = Ys[:, i:i+batch_size]

            # forward
            A = Xb
            for layer in layers:
                A = layer.forward(A)

            # loss + backward
            loss = loss_fn.forward(A, Yb)
            dA = loss_fn.backward(A, Yb)

            # backprop
            for layer in reversed(layers):
                dA, dW, db = layer.backward(dA)
                optimizer.update(layer, dW, db)

    # ---- compute validation accuracy ----
    A = X_val_fold.T
    for layer in layers:
        A = layer.forward(A)

    val_acc = accuracy_from_probs(A, Y_val_fold)
    return val_acc


from sklearn.model_selection import KFold

def train_on_split(X_train_fold, Y_train_fold, X_val_fold, Y_val_fold, cfg):
    """
    Train a model on one train/val split.
    Returns final validation accuracy.
    """

    layers = build_layers(X_train_fold.shape[1], cfg)
    optimizer = get_optimizer(cfg['optimizer'], float(cfg['learning_rate']))
    loss_fn = CrossEntropyLoss()

    m = X_train_fold.shape[0]
    epochs = int(cfg["epochs"])
    batch_size = int(cfg["batch_size"])

    for epoch in range(epochs):
        perm = np.random.permutation(m)

        Xs = X_train_fold[perm].T     # (in_dim, m)
        Ys = Y_train_fold[:, perm]    # (num_classes, m)

        for i in range(0, m, batch_size):
            Xb = Xs[:, i:i+batch_size]
            Yb = Ys[:, i:i+batch_size]

            # forward
            A = Xb
            for layer in layers:
                A = layer.forward(A)

            # loss + backward
            loss = loss_fn.forward(A, Yb)
            dA = loss_fn.backward(A, Yb)

            # backprop
            for layer in reversed(layers):
                dA, dW, db = layer.backward(dA)
                optimizer.update(layer, dW, db)

    # ---- compute validation accuracy ----
    A = X_val_fold.T
    for layer in layers:
        A = layer.forward(A)

    val_acc = accuracy_from_probs(A, Y_val_fold)
    return val_acc

def train_run():

    with wandb.init() as run:
        cfg = dict(run.config)
        run.log(dict(cfg))

        kf = KFold(n_splits=3, shuffle=True, random_state=42)
        fold_accuracies = []

        # K-fold loop
        for fold_idx, (train_idx, val_idx) in enumerate(kf.split(X_train)):

            X_train_fold = X_train[train_idx]
            Y_train_fold = Y_train[:, train_idx]

            X_val_fold = X_train[val_idx]
            Y_val_fold = Y_train[:, val_idx]

            val_acc = train_on_split(
                X_train_fold, Y_train_fold,
                X_val_fold, Y_val_fold,
                cfg
            )

            fold_accuracies.append(val_acc)
            run.log({f"fold_{fold_idx+1}_accuracy": float(val_acc)})

        # final metric for sweep
        mean_cv_acc = float(np.mean(fold_accuracies))
        run.log({"mean_cv_accuracy": mean_cv_acc})

        print(f"Run {run.id}  CV Accuracies={fold_accuracies}  Mean={mean_cv_acc:.4f}")

        wandb.finish()

wandb.agent(sweep_id, function=train_run, count=10)

Create sweep with ID: rhd00nb0
Sweep URL: https://wandb.ai/xanderbaatz-danmarks-tekniske-universitet-dtu/numpy_nn/sweeps/rhd00nb0


wandb: Agent Starting Run: 14sqivwa with config:
wandb: 	activation: sigmoid
wandb: 	batch_size: 32
wandb: 	epochs: 30
wandb: 	l2_coeff: 0.0008442830132521463
wandb: 	learning_rate: 0.5917482247650961
wandb: 	n_hidden_units: 110
wandb: 	num_classes: 3
wandb: 	num_hidden_layers: 2
wandb: 	optimizer: adam
wandb: Currently logged in as: glymse (xanderbaatz-danmarks-tekniske-universitet-dtu) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


/workspaces/02456-deep-learning/NN/activations.py:25: RuntimeWarning: overflow encountered in exp
  self.A = 1 / (1 + np.exp(-Z))
wandb: ERROR The nbformat package was not found. It is required to save notebook history.


Run 14sqivwa  CV Accuracies=[0.34, 0.35, 0.343]  Mean=0.3443


batch_size,▁
epochs,▁
fold_1_accuracy,▁
fold_2_accuracy,▁
fold_3_accuracy,▁
l2_coeff,▁
learning_rate,▁
mean_cv_accuracy,▁
n_hidden_units,▁
num_classes,▁
+1,...


...

Run 2bai9irb  CV Accuracies=[0.34, 0.333, 0.327]  Mean=0.3333


batch_size,▁
epochs,▁
fold_1_accuracy,▁
fold_2_accuracy,▁
fold_3_accuracy,▁
l2_coeff,▁
learning_rate,▁
mean_cv_accuracy,▁
n_hidden_units,▁
num_classes,▁
+1,...


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: 4ax40jlk with config:
wandb: 	activation: relu
wandb: 	batch_size: 128
wandb: 	epochs: 30
wandb: 	l2_coeff: 0.0003928648513476744
wandb: 	learning_rate: 0.572117577466804
wandb: 	n_hidden_units: 55
wandb: 	num_classes: 3
wandb: 	num_hidden_layers: 2
wandb: 	optimizer: sgd


wandb: Ctrl + C detected. Stopping sweep.


Traceback (most recent call last):
  File "/tmp/ipykernel_79893/424036661.py", line 179, in train_run
    run.log({f"fold_{fold_idx+1}_accuracy": float(val_acc)})
  File "/home/vscode/.local/lib/python3.10/site-packages/wandb/sdk/wandb_run.py", line 390, in wrapper
    return func(self, *args, **kwargs)
  File "/home/vscode/.local/lib/python3.10/site-packages/wandb/sdk/wandb_run.py", line 448, in wrapper_fn
    return func(self, *args, **kwargs)
  File "/home/vscode/.local/lib/python3.10/site-packages/wandb/sdk/wandb_run.py", line 435, in wrapper
    return func(self, *args, **kwargs)
  File "/home/vscode/.local/lib/python3.10/site-packages/wandb/sdk/wandb_run.py", line 2023, in log
    self._log(data=data, step=step, commit=commit)
  File "/home/vscode/.local/lib/python3.10/site-packages/wandb/sdk/wandb_run.py", line 1734, in _log
    self._partial_history_callback(data, step, commit)
  File "/home/vscode/.local/lib/python3.10/site-packages/wandb/sdk/wandb_run.py", line 390, in wrappe